In [23]:
from dash import Dash

import pandas as pd



# I fixed your imports for dash.dcc and dash.html,  the reason you were getting a warning message was because the way you imported the library worked

# but was an outdated method of doing it



from dash import dcc,html

import plotly.express as px

from dash.dependencies import Input, Output

import pymssql

import plotly.graph_objects as go




import warnings # The method used in class gives a warning when using pd.read_sql, this will ignore the warning message

warnings.simplefilter(action='ignore', category=UserWarning)



database = "Pushing-P-DB"

table = "dbo.NJ_Cities"

user = "pushing_p"

password  = "t3stP@ssword"

server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

# Make function to retrieve data from database

def Get_Data():

    try:

        conn = pymssql.connect(server,user,password,database)



        query = f"SELECT * FROM {table}"



        df = pd.read_sql(query, conn)



        # converting to pandas dataframe

        df = pd.DataFrame(df)

        df['Rate Of Uninsured'] = (df['Uninsured Population']/df['City Population'])

        df['Rate Of Uninsured'] = df['Rate Of Uninsured'].round(decimals = 5)

        df['Insured Population'] = df['City Population'] - df['Uninsured Population']

        df['Insured Population'] = df['Insured Population'].round(decimals = 5)

        df['Rate Of Insured'] = df['Insured Population'] / df['City Population']

        df['Rate Of Insured'] = df['Rate Of Insured'].round(decimals = 5)



        # This is where I format the TimeStamp to "ms" and then I have to for some reason use ".dt.strftime('%Y-%m-%d %H:%M:%S')" because the format in the table chances when I print it out

#         df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], unit='ms').dt.strftime('%Y-%m-%d %H:%M:%S')



        # This is where I order based off of timestamp for my fifth dataframe

#         df = df.sort_values(by='TimeStamp',ascending=False)



        return df



    except Exception as e:

        raise e




# Here I set the colors I will be using for the visualizations

colors = {

    'background': '#111111',

    'text': '#7FDBFF'

}

In [24]:
df = Get_Data()
df


,State,County,City,Unemployment Rate (16 & Over),Median Household Income,City Population,Uninsured Population,"NUI, Under 6 years","NUI, 6-18 years","NUI, 19 to 25 years",...,"NUI, White","NUI, African American","NUI, American Indian & Alaska Native Population","NUI, Asians","NUI, Native Hawaiians & Pacific Islanders Population","NUI, Other Races",geometry,Rate Of Uninsured,Insured Population,Rate Of Insured
0,New Jersey,Burlington County,Riverside Township,9.8,61975,7836,1409,17,85,202,...,821,154,28,13,0,236,01030000000100000048000000B60F79CBD5B752C0F182...,0.17981,6427,0.82019
1,New Jersey,Monmouth County,Manalapan Township,4.1,130663,39462,950,0,114,101,...,734,28,0,143,0,17,010300000002000000650000008E75711B0D9C52C00725...,0.02407,38512,0.97593
2,New Jersey,Monmouth County,Manasquan Borough,4.9,116435,5837,113,0,17,0,...,113,0,0,0,0,0,01030000000100000015000000641F645930B352C019E1...,0.01936,5724,0.98064
3,New Jersey,Burlington County,Riverton Borough,5.1,100278,2592,75,12,0,0,...,42,14,0,0,0,19,01030000000100000045000000B64604E3E0B452C0D445...,0.02894,2517,0.97106
4,New Jersey,Monmouth County,Fair Haven Borough,3.2,184201,5833,122,0,34,0,...,121,1,0,0,0,0,01030000000100000015000000C2BF081A338052C062BA...,0.02092,5711,0.97908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,New Jersey,Bergen County,Englewood,5.9,86485,28213,2735,121,407,267,...,1514,698,34,17,0,411,0103000000010000002D000000C26B97361C8052C01616...,0.09694,25478,0.90306
561,New Jersey,Bergen County,Englewood Cliffs Borough,4.6,158042,5349,252,0,9,0,...,138,0,0,92,10,0,010300000001000000310000001F63EE5A42A852C063B3...,0.04711,5097,0.95289
562,New Jersey,Monmouth County,Englishtown Borough,9.0,84091,1999,51,0,0,10,...,25,16,0,0,0,0,0103000000010000002E000000471E882CD29952C0FF76...,0.02551,1948,0.97449
563,New Jersey,Essex County,Essex Fells Borough,3.1,215104,2037,15,0,4,0,...,15,0,0,0,0,0,01030000000100000017000000B06EBC3B32C452C01022...,0.00736,2022,0.99264


In [25]:
data = df[['City','City Population','Uninsured Population']]

In [26]:
data

,City,City Population,Uninsured Population
0,Riverside Township,7836,1409
1,Manalapan Township,39462,950
2,Manasquan Borough,5837,113
3,Riverton Borough,2592,75
4,Fair Haven Borough,5833,122
...,...,...,...
560,Englewood,28213,2735
561,Englewood Cliffs Borough,5349,252
562,Englishtown Borough,1999,51
563,Essex Fells Borough,2037,15


In [46]:
x = data['City Population'].sum()
y = data['Uninsured Population'].sum()
print('Uninsured Population {}%'.format(y*100/x))

Uninsured Population 7.550169928566222%


In [47]:
import plotly.graph_objects as go

labels = ['Uninsured Population','City Population']
values = [y,x]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()